In [47]:
import numpy as np
import lxml.etree as ET
import plotly.express as px
import pandas as pd
from typing import List, Dict

# tree = ET.parse("../recordings/c.xml")
tree = ET.parse(
    "/home/yiguo/Desktop/apixC_MC七日杀/footages/47867_20230329_044736_打新的女神异闻录，MC.xml"
)
root = tree.getroot()


In [48]:
info = {}
danmaku = []
for child in root:
    if child.tag == "BililiveRecorderRecordInfo":
        info = child.attrib
        print(child.text)
    elif child.tag == "d":
        danmaku.append(
            {
                "time": child.attrib["p"].split(",")[0],
                "user": child.attrib["user"],
                "text": child.text,
            }
        )

for key in info:
    print(key + ": " + info[key])
for d in danmaku[:0]:
    print(d)
print("...")


None
roomid: 47867
shortid: 213
name: C酱です
title: 打新的女神异闻录，MC
areanameparent: 单机游戏
areanamechild: 主机游戏
start_time: 2023-03-29T04:47:36.9751889-07:00
...


In [3]:
class Keyword:
    def __init__(self, exact_match_list, fuzzy_match_list, weight_factor):
        self.exact_match_list = exact_match_list
        self.fuzzy_match_list = fuzzy_match_list
        self.weight_factor = weight_factor

    def match(self, text):
        if text in self.exact_match_list:
            return True
        for keyword in self.fuzzy_match_list:
            if keyword in text:
                return True
        return False


In [24]:
danmaku_whitelist = {
    # "global": Keyword([], [''], 1),
    # "搜索": Keyword(["挂","有挂","呱","有呱","准","好准","挂!","有挂!","呱!","有呱!","准!","好准!"],[],100),
    '1': Keyword(['1'], [], 1),
    # "？": Keyword(["？", "?"], ["？？", "??", "问号"], 2),
    # "乐": Keyword(["乐"], ["哈哈", "233", "笑死"], 1),
    # "草": Keyword([], ["草", "艹", "cao"], 1),
    # "甜": Keyword([], ["甜", "kdl", "磕", "ttk"], 4),
    # "xx": Keyword([], ["xx"], 1),
    # "酷": Keyword([], ["酷"], 1),
    # "三不离": Keyword([], ["三不离", "三句不离"], 4),
    # "不好说": Keyword([], ["不好说", "暗恋"], 4),
    # "典": Keyword([], ["血口喷人", "我会那么笨吗", "内心的想法", "攻击性","啊对对对","头皮发麻","易大山","睡不着了"], 4),
    # "两眼": Keyword([], ["两眼", "眼前", "双眼", "双目"], 4),
    # "逆天": Keyword([], ["逆天", "变态"], 2),
    # "汗": Keyword([], ["汗"], 2),
    # "可爱": Keyword([], ["可爱"], 4),
    # "好": Keyword(["好"], ["好好", "好耶", "棒"], 1),
    # "确实": Keyword([], ["确实", "正确", "合理"], 1),
    # "坏了": Keyword(["坏了"],[],1),
    # "小肉包": Keyword([], ["小肉包", "肉包", "同事", "C酱", "c酱", "双马尾", "闪闪发光", "非常漂亮"], 1),
    # "api": Keyword([], ["pi", "Pi", "PI"], 1),
    # "切片man": Keyword([], ["切片man"], 10),
}


In [49]:
p = {}
time_list_tot = []
for d in danmaku:
    text = d["text"]
    time = round(float(d["time"]))
    weight = 1.0
    # filter and categorize danmaku
    valid = False
    for key in danmaku_whitelist:
        keyword = danmaku_whitelist[key]
        if keyword.match(text):
            text = key
            weight *= keyword.weight_factor
            valid = True
            break
    if not valid:
        continue
    # add danmaku to pool
    item = {"time": time, "weight": weight}
    if text in p:
        p[text].append(item)
    else:
        p[text] = [item]
    # add time to total time list
    time_list_tot.append(item)

time_list_text = p
time_list_text_sorted = [{"text": text, "time_list": p[text]} for text in p]


def sort_key(e: Dict[str, any]):
    return len(e["time_list"])


time_list_text_sorted.sort(reverse=True, key=sort_key)

for item in time_list_text_sorted:
    print(item["text"], len(item["time_list"]))



1 665


In [50]:
def calc_freq_map(time_list: List[int], max_time: int):
    average_duration = 15
    map_step_size = 5

    # the input list should have been already in ascending order, but we make sure about it again
    sort_key = lambda e: e["time"]
    time_list.sort(key=sort_key)
    # calculate the time averaged frequency
    x = [_ for _ in range(0, max_time, map_step_size)]
    y = [0 for _ in range(0, max_time, map_step_size)]
    hi, lo = 0, 0
    max_index = len(time_list) - 1
    for i in range(len(x)):
        left = x[i] - average_duration / 2
        right = x[i] + average_duration / 2
        while time_list[lo]["time"] < left and lo < max_index:
            lo += 1
        while time_list[hi]["time"] < right and hi < max_index:
            hi += 1
        tot = 0
        for j in range(lo, hi):
            tot += time_list[j]["weight"]
        y[i] = tot / average_duration

    return x, y


In [51]:
max_time = 0
tot = 0
for item in time_list_tot:
    if item["time"] > max_time:
        max_time = item["time"]
    tot += item["weight"]
average_frequency = tot / max_time

print("max time:", max_time)
print("average frequency:", average_frequency)


max time: 20424
average frequency: 0.03255973364669017


In [52]:
threshold = max(3 * average_frequency, 2)
backward_time = 60
afterward_time = 30

data = {}
for key in time_list_text:
    x, y = calc_freq_map(time_list_text[key], max_time)
    data[key] = y

hotspot_list = []
x, y = calc_freq_map(time_list_tot, max_time)
for i in range(1, len(x) - 1):
    if y[i] > threshold and y[i] >= y[i - 1] and y[i] >= y[i + 1]:
        start = x[i] - backward_time
        end = x[i] + afterward_time
        hotspot = {
            "time": x[i],
            "start": start if start > 0 else 0,
            "end": end if end < max_time else max_time,
        }
        if len(hotspot_list) > 0 and hotspot_list[-1]["end"] > hotspot["start"]:
            hotspot_list[-1]["end"] = hotspot["end"]
        else:
            hotspot_list.append(hotspot)

tot = 0
for item in hotspot_list:
    tot += item["end"] - item["start"]
print("Number of clips:", len(hotspot_list))
print("Clip total length:", tot)
print("Compress ratio: " +  f"{max_time / tot:.2f}")


Number of clips: 0
Clip total length: 0


ZeroDivisionError: division by zero

In [22]:
def sec2time(sec: int):
    return f"{sec // 3600}:{(sec % 3600) // 60}:{sec % 60}"


In [54]:
data = []
# add threshold line
x, y = calc_freq_map(time_list_tot, max_time)
for i in range(len(x)):
    data.append(["threshold", sec2time(x[i]), threshold])
# add total frequency
for i in range(len(x)):
    data.append(["All", sec2time(x[i]), y[i]])
# add sub frequencies
for item in time_list_text_sorted:
    text = item["text"]
    x, y = calc_freq_map(time_list_text[text], max_time)
    for i in range(len(x)):
        data.append([text, sec2time(x[i]), y[i]])
# add clip coverages
for item in hotspot_list:
    for x in [item["start"], item["end"]]:
        data.append([str(item["time"]), sec2time(x), threshold + 1])
# export to dataframe
df = pd.DataFrame(data, columns=["Danmaku", "Time", "Frequency"])

fig = px.line(df, x="Time", y="Frequency", color="Danmaku")
fig.show()


In [11]:
data = []
x, y = calc_freq_map(time_list_tot, max_time)
for i in range(len(x)):
    data.append([text, x[i], y[i]])
df = pd.DataFrame(data, columns=["Danmaku", "Time", "Frequency"])

fig = px.histogram(df, x="Frequency")
fig.show()
